In [67]:
import time

from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(message)s',level=logging.DEBUG)

from elasticsearch import Elasticsearch
from clickhouse_driver import Client

class TestRunner:
    def __init__(self, name, config):
        self.name = name
        self.config = config
    
    def run(self, case):
        pass

class ElasticsearchRuner(TestRunner):
    def __init__(self, name, config):
        TestRunner.__init__(self, name, config)
        self.client = Elasticsearch([self.config['cluster']])
        self.indices = self.config['indices']
           
    def run(self, case):
        query = case
        
        logging.info(f'{self.name} run query {case}')
        start = time.time()
        response = self.client.search(index=self.indices, body=query)
        end = time.time()
        logging.info(f'{self.name} run query complete')
        
        result = {}
        result['count'] = response['hits']['total']['value']
        result['response'] = response
        result['elapsed'] = end - start
        
        logging.info(f'query result count {result["count"]} elapsed {result["elapsed"]}')
        return result
        
class ClickhouseRuner(TestRunner):
    def __init__(self, name, config):
        TestRunner.__init__(self, name, config)
        self.client = Client(self.config['cluster'])
    
    def run(self, case):
        query = case
        logging.info(f'{self.name} run query {case}')
        start = time.time()
        response = self.client.execute(query)
        end = time.time()
        logging.info(f'{self.name} run query complete')
        
        result = {}
        result['count'] = len(response)
        result['response'] = response
        result['elapsed'] = end - start
        
        logging.info(f'query result count {result["count"]} elapsed {result["elapsed"]}')
        return result

In [65]:
clickhouse_config = { "cluster" : "host.docker.internal"} 
clickhouse_runner = ClickhouseRuner('clickhouse', clickhouse_config)

query = 'SELECT * FROM syslog'
clickhouse_runner.run(query)

2021-02-25 18:24:15,934 clickhouse run query SELECT * FROM syslog
2021-02-25 18:24:15,971 clickhouse run query complete
2021-02-25 18:24:15,972 query result count 2000 elapsed 0.0353543758392334


{'count': 2000,
 'response': [('meln1ks',
   'random.us',
   "A bug was encountered but not in Vector, which doesn't have bugs",
   'ID849',
   '7516',
   49,
   "<49>2 2021-02-24T23:46:04.415Z random.us meln1ks 7516 ID849 - A bug was encountered but not in Vector, which doesn't have bugs",
   datetime.datetime(2021, 2, 24, 23, 46, 4, tzinfo=<UTC>),
   2),
  ('Karimmove',
   'we.net',
   "You're not gonna believe what just happened",
   'ID997',
   '8646',
   113,
   "<113>2 2021-02-24T23:46:04.425Z we.net Karimmove 8646 ID997 - You're not gonna believe what just happened",
   datetime.datetime(2021, 2, 24, 23, 46, 4, tzinfo=<UTC>),
   2),
  ('devankoshal',
   'for.org',
   "Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!",
   'ID216',
   '3366',
   154,
   "<154>1 2021-02-24T23:46:04.435Z for.org devankoshal 3366 ID216 - Great Scott! We're never gonna reach 88 mph with the flux capacitor in its current state!",
   datetime.datetime(2021, 2, 24

In [66]:
es_config = { "cluster" : "host.docker.internal", "indices" : "syslog-2021-02-24,syslog-2021-02-25"} 
es_runner = ElasticsearchRuner('es', es_config)

query = {
    "query": {
        "match_all": {}
    }
}
es_runner.run(query)

2021-02-25 18:24:22,393 es run query {'query': {'match_all': {}}}
2021-02-25 18:24:22,404 es run query complete
2021-02-25 18:24:22,405 query result count 2000 elapsed 0.009815454483032227


{'count': 2000,
 'response': {'took': 1,
  'timed_out': False,
  '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 2000, 'relation': 'eq'},
   'max_score': 1.0,
   'hits': [{'_index': 'syslog-2021-02-24',
     '_type': '_doc',
     '_id': 'iNVu1ncBIYMyHW4dba-j',
     '_score': 1.0,
     '_source': {'application': 'ahmadajmi',
      'hostname': 'random.net',
      'message': 'Pretty pretty pretty good',
      'mid': 'ID3',
      'pid': '4895',
      'priority': 117,
      'raw': '<117>2 2021-02-24T23:44:56.160Z random.net ahmadajmi 4895 ID3 - Pretty pretty pretty good',
      'timestamp': '2021-02-24T23:44:56.160Z',
      'version': 2}},
    {'_index': 'syslog-2021-02-24',
     '_type': '_doc',
     '_id': 'idVu1ncBIYMyHW4dba-j',
     '_score': 1.0,
     '_source': {'application': 'devankoshal',
      'hostname': 'make.de',
      'message': "You're not gonna believe what just happened",
      'mid': 'ID51',
      'pid': '9927',
      'pri

In [71]:
import json

class TestPlan:
    def __init__(self, planfile):
        with open(planfile) as json_file:
            self.plan = json.load(json_file)
            self.targets = []
            for target in self.plan['targets']:
                if target['type'] == 'elasticsearch':
                    runner = ElasticsearchRuner(target['name'], target['config'])
                    self.targets.append(runner)
                elif target['type'] == 'clickhouse':
                    runner = ClickhouseRuner(target['name'], target['config'])
                    self.targets.append(runner)
    
    def run(self):
        reports = []
        for testcase in self.plan['testcases']:
            name, description, cases = testcase
            report = {}
            report['name'] = name
            report['description'] = description
            report['result'] = []
            logging.info(f'run testcase {name} {description}')
            tests = list(zip(self.targets, cases))
            for test in tests:
                runner, query = test
                result = {}
                result['target'] = runner.name
                result['case'] = str(query)
                
                elapsed = 0
                for i in range(self.plan['runs']):
                    run_result = runner.run(query)
                    elapsed += run_result['elapsed']
                    
                result['elapsed'] = elapsed
                report['result'].append(result)
                
            logging.info(f'run testcase {name} complete')
            reports.append(report)
        return reports


testplan = TestPlan("testplan.json")
reports = testplan.run()

2021-02-25 18:29:45,386 run testcase case1 query all
2021-02-25 18:29:45,387 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,402 es1 run query complete
2021-02-25 18:29:45,403 query result count 2000 elapsed 0.012495279312133789
2021-02-25 18:29:45,408 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,416 es1 run query complete
2021-02-25 18:29:45,417 query result count 2000 elapsed 0.006368875503540039
2021-02-25 18:29:45,420 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,426 es1 run query complete
2021-02-25 18:29:45,427 query result count 2000 elapsed 0.005052328109741211
2021-02-25 18:29:45,428 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,434 es1 run query complete
2021-02-25 18:29:45,436 query result count 2000 elapsed 0.005082845687866211
2021-02-25 18:29:45,437 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,446 es1 run query complete
2021-02-25 18:29:45,447 query result count 2000 elapsed 0.00721406936

2021-02-25 18:29:45,838 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,843 es1 run query complete
2021-02-25 18:29:45,845 query result count 2000 elapsed 0.004182577133178711
2021-02-25 18:29:45,846 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,852 es1 run query complete
2021-02-25 18:29:45,854 query result count 2000 elapsed 0.004723548889160156
2021-02-25 18:29:45,855 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,861 es1 run query complete
2021-02-25 18:29:45,862 query result count 2000 elapsed 0.004245758056640625
2021-02-25 18:29:45,863 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,869 es1 run query complete
2021-02-25 18:29:45,870 query result count 2000 elapsed 0.004919290542602539
2021-02-25 18:29:45,871 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:45,877 es1 run query complete
2021-02-25 18:29:45,878 query result count 2000 elapsed 0.00414276123046875
2021-02-25 18:29:45,879 es1 run query {'query'

2021-02-25 18:29:46,128 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:46,134 es1 run query complete
2021-02-25 18:29:46,135 query result count 2000 elapsed 0.005174398422241211
2021-02-25 18:29:46,136 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:46,141 es1 run query complete
2021-02-25 18:29:46,142 query result count 2000 elapsed 0.005072593688964844
2021-02-25 18:29:46,143 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:46,149 es1 run query complete
2021-02-25 18:29:46,150 query result count 2000 elapsed 0.005090475082397461
2021-02-25 18:29:46,151 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:46,157 es1 run query complete
2021-02-25 18:29:46,158 query result count 2000 elapsed 0.005100250244140625
2021-02-25 18:29:46,159 es1 run query {'query': {'match_all': {}}}
2021-02-25 18:29:46,164 es1 run query complete
2021-02-25 18:29:46,165 query result count 2000 elapsed 0.00487208366394043
2021-02-25 18:29:46,166 es1 run query {'query'

2021-02-25 18:29:46,991 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:47,012 clickhouse1 run query complete
2021-02-25 18:29:47,013 query result count 2000 elapsed 0.020025968551635742
2021-02-25 18:29:47,015 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:47,038 clickhouse1 run query complete
2021-02-25 18:29:47,039 query result count 2000 elapsed 0.021546602249145508
2021-02-25 18:29:47,041 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:47,066 clickhouse1 run query complete
2021-02-25 18:29:47,067 query result count 2000 elapsed 0.02319192886352539
2021-02-25 18:29:47,069 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:47,092 clickhouse1 run query complete
2021-02-25 18:29:47,093 query result count 2000 elapsed 0.022432565689086914
2021-02-25 18:29:47,096 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:47,118 clickhouse1 run query complete
2021-02-25 18:29:47,119 query result count 2000 elapsed 0.02146768569946289
2021-02

2021-02-25 18:29:48,122 clickhouse1 run query complete
2021-02-25 18:29:48,124 query result count 2000 elapsed 0.020256519317626953
2021-02-25 18:29:48,126 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:48,149 clickhouse1 run query complete
2021-02-25 18:29:48,150 query result count 2000 elapsed 0.022195100784301758
2021-02-25 18:29:48,152 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:48,174 clickhouse1 run query complete
2021-02-25 18:29:48,174 query result count 2000 elapsed 0.020789146423339844
2021-02-25 18:29:48,176 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:48,198 clickhouse1 run query complete
2021-02-25 18:29:48,199 query result count 2000 elapsed 0.021021604537963867
2021-02-25 18:29:48,200 clickhouse1 run query SELECT * FROM syslog
2021-02-25 18:29:48,220 clickhouse1 run query complete
2021-02-25 18:29:48,222 query result count 2000 elapsed 0.01890087127685547
2021-02-25 18:29:48,223 clickhouse1 run query SELECT * FROM syslog
2021-0

In [72]:
reports

[{'name': 'case1',
  'description': 'query all',
  'result': [{'target': 'es1',
    'case': "{'query': {'match_all': {}}}",
    'elapsed': 0.48037219047546387},
   {'target': 'clickhouse1',
    'case': 'SELECT * FROM syslog',
    'elapsed': 2.3805699348449707}]}]